# Numerical Solution


In [1]:
using CairoMakie
using ModelingToolkit
using DifferentialEquations: solve

┌ Warning: The call to compilecache failed to create a usable precompiled cache file for LogExpFunctions [2ab3a3ac-af41-5b50-aa03-7779005ae688]
│   exception = Required dependency ChangesOfVariables [9e997f8a-9a97-42d5-a9f1-ce6bfc15e2c0] failed to load from a cache file.
└ @ Base loading.jl:1132
┌ Warning: Package ChangesOfVariables does not have ChainRulesCore in its dependencies:
│ - If you have ChangesOfVariables checked out for development and have
│   added ChainRulesCore as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ChangesOfVariables
└ Loading ChainRulesCore into ChangesOfVariables from project dependency, future warnings for ChangesOfVariables are suppressed.
┌ Warning: The call to compilecache failed to create a usable precompiled cache file for ColorVectorSpace [c3611d14-8923-5661-9e6a-0046d554d3a4]
│   exception = Required dependency ChangesOfVariables [9e997f8a-9a97-4

┌ Warning: The call to compilecache failed to create a usable precompiled cache file for LogExpFunctions [2ab3a3ac-af41-5b50-aa03-7779005ae688]
│   exception = Required dependency ChangesOfVariables [9e997f8a-9a97-42d5-a9f1-ce6bfc15e2c0] failed to load from a cache file.
└ @ Base loading.jl:1132
┌ Warning: Package ChangesOfVariables does not have ChainRulesCore in its dependencies:
│ - If you have ChangesOfVariables checked out for development and have
│   added ChainRulesCore as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ChangesOfVariables
└ Loading ChainRulesCore into ChangesOfVariables from project dependency, future warnings for ChangesOfVariables are suppressed.
┌ Warning: The call to compilecache failed to create a usable precompiled cache file for SpecialFunctions [276daf66-3868-5448-9aa4-cd146d93841b]
│   exception = Required dependency ChangesOfVariables [9e997f8a-9a97-4

┌ Warning: Package ChangesOfVariables does not have ChainRulesCore in its dependencies:
│ - If you have ChangesOfVariables checked out for development and have
│   added ChainRulesCore as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ChangesOfVariables
└ Loading ChainRulesCore into ChangesOfVariables from project dependency, future warnings for ChangesOfVariables are suppressed.
┌ Warning: unknown command \Join
└ @ MathTeXEngine ~/.julia/packages/MathTeXEngine/ZP0gS/src/parser/commands_registration.jl:48
┌ Warning: Module LogExpFunctions with build ID 356965076977214 is missing from the cache.
│ This may mean LogExpFunctions [2ab3a3ac-af41-5b50-aa03-7779005ae688] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1107
┌ Warning: Module StatsBase with build ID 356971698780815 is missing from the cache.
│ This may mean StatsBase [2913bbd2-ae8a-5

┌ Warning: The call to compilecache failed to create a usable precompiled cache file for FreeTypeAbstraction [663a7486-cb36-511b-a19d-713bb74d65c9]
│   exception = Required dependency ChangesOfVariables [9e997f8a-9a97-42d5-a9f1-ce6bfc15e2c0] failed to load from a cache file.
└ @ Base loading.jl:1132
┌ Warning: Package ChangesOfVariables does not have ChainRulesCore in its dependencies:
│ - If you have ChangesOfVariables checked out for development and have
│   added ChainRulesCore as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ChangesOfVariables
└ Loading ChainRulesCore into ChangesOfVariables from project dependency, future warnings for ChangesOfVariables are suppressed.
┌ Warning: The call to compilecache failed to create a usable precompiled cache file for LogExpFunctions [2ab3a3ac-af41-5b50-aa03-7779005ae688]
│   exception = Required dependency ChangesOfVariables [9e997f8a-9a9

┌ Warning: The call to compilecache failed to create a usable precompiled cache file for SpecialFunctions [276daf66-3868-5448-9aa4-cd146d93841b]
│   exception = Required dependency ChangesOfVariables [9e997f8a-9a97-42d5-a9f1-ce6bfc15e2c0] failed to load from a cache file.
└ @ Base loading.jl:1132
┌ Warning: Package ChangesOfVariables does not have ChainRulesCore in its dependencies:
│ - If you have ChangesOfVariables checked out for development and have
│   added ChainRulesCore as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ChangesOfVariables
└ Loading ChainRulesCore into ChangesOfVariables from project dependency, future warnings for ChangesOfVariables are suppressed.
┌ Warning: The call to compilecache failed to create a usable precompiled cache file for LogExpFunctions [2ab3a3ac-af41-5b50-aa03-7779005ae688]
│   exception = Required dependency ChangesOfVariables [9e997f8a-9a97-4

## Euler Methods


In [ ]:
function euler_forward(t, n_steps, y₀)
    f = zeros(size(t))
    y = zeros(size(t))
    y[1] = y₀
    y[2] = f[end-1]

    for ii = 1:n_steps
        f[ii] = 3 + exp(-t[ii]) - 2y[ii]
        y[ii+1] = y[ii] + f[ii] * h
    end
    return y
end

function euler_improved(t, n_steps, y₀)
    f = zeros(size(t))
    f[1] = 3 + exp(-t[1]) - 2y₀
    y = zeros(size(t))
    y[1] = y₀
    for ii = 1:n_steps
        f[ii+1] = 3 + exp(-(t[ii] + h)) - 2(y[ii] + h * f[ii])
        y[ii+1] = y[ii] + (f[ii] + f[ii+1]) * (h / 2)
    end
    return y
end

function euler_backward(T, n_steps, y₀)
    f = zeros(n_steps + 1)
    y_new = zeros(n_steps + 1)
    y = zeros(n_steps + 1)
    y[1] = y₀

    for ii = 1:n_steps
        T[ii+1] = T[ii] + h
        f[ii] = 3 - 2y[ii] + exp(-T[ii])
        y_new[ii] = y[ii] + h * f[ii]
        f[ii+1] = 3 - 2y_new[ii] + exp(-T[ii+1])
        y[ii+1] = y[ii] + f[ii+1] * h
    end
    return y
end

In [ ]:
t₀ = 0;
tₙ = 10;
h = 0.05;
t = t₀:h:tₙ;
n_steps = length(t) - 1;
y₀ = 1;

y₁ = euler_forward(t, n_steps, y₀);
y₂ = euler_improved(t, n_steps, y₀);

T = zeros(n_steps + 1);
y₃ = euler_backward(T, n_steps, y₀);

colors = [:red :green :orange];
labels = ["Euler-Forward", "Euler-Improved", "Euler-Backward"];

fig = Figure(resolution = (600, 400))
ax = Axis(fig[1, 1], xlabel = "t", ylabel = "y(t)")

lines!(ax, t, y₁, color = colors[1], linewidth = 1.5, label = labels[1])
lines!(ax, t, y₂, color = colors[2], linewidth = 1.5, label = labels[2])
lines!(ax, T, y₃, color = colors[3], linewidth = 1.5, label = labels[3])
axislegend(ax; position = :rb, labelsize = 12)
fig

In [ ]:
h = 0.2; # time step size()
t₀ = 0; # Start of simulations
tₙ = 7; # End of simulations
t = t₀:h:tₙ
n_steps = length(t);
u = zeros((n_steps, 2))
u[1, :] = [1.0, 2.0]

for ii = 2:n_steps
    f = [u[ii-1, 2], 4t[ii] - (4 / 5) * u[ii-1, 2] - 2u[ii-1, 1]]
    u[ii, :] = u[ii-1, :] + f * h
end

fig = Figure(resolution = (600, 400))
ax = Axis(
    fig[1, 1],
    xlabel = "t",
    ylabel = "Solution, y(t)",
    title = "Numerical Solutions of: 1/2u″ + 2/5u′ + u = 2t",
)

lines!(ax, t, u[:, 1], label = "Euler-Forward")
axislegend(ax, position = :rb, labelsize = 12)
fig

## Runge Kutta Methods


In [ ]:
function runge_kutta(t, n_steps, y₀)
    k₁ = zeros(1, n_steps)
    k₂ = k₁
    k₃ = k₁
    k₄ = k₁
    y = zeros(n_steps + 1)
    y[1] = y₀

    for ii = 1:n_steps
        k₁[ii] = 3 + exp(-t[ii]) - 2y[ii]
        k₂[ii] = 3 + exp(-(t[ii] + h / 2)) - 2(y[ii] + k₁[ii] * h / 2)
        k₃[ii] = 3 + exp(-(t[ii] + h / 2)) - 2(y[ii] + k₂[ii] * h / 2)
        k₄[ii] = 3 + exp(-(t[ii] + h)) - 2(y[ii] + k₃[ii] * h)
        y[ii+1] = y[ii] + h * (k₁[ii] + 2k₂[ii] + 2k₃[ii] + k₄[ii]) / 6
    end
    return y
end

In [ ]:
@variables ts ys(ts)
@parameters e

D = Differential(ts)
@named runge_kutta_s = ODESystem(D(ys) ~ -2ys + e^(-ts) + 3)

prob = ODEProblem(runge_kutta_s, [ys => 1.0], (0.0, 10.0), [e => ℯ])
sol = solve(prob);

In [ ]:
t₀ = 0;
tₙ = 10;
h = 0.05;
t = t₀:h:tₙ;
n_steps = length(t) - 1;
y₀ = 1;

y = runge_kutta(t, n_steps, y₀)

fig = Figure(resolution = (600, 400))
ax = Axis(
    fig[1, 1],
    xlabel = "t",
    ylabel = "Solution, y(t)",
    title = "Numerical Solutions of: y′ + 2y - e⁻ᵗ = 3, y₀ = 1",
)

lines!(ax, t, y, linewidth = 1.5, label = "Runge-Kutta Method")
lines!(ax, sol.t, vcat(sol.u...), linewidth = 1.5, label = "Analytic solution")
axislegend(ax, position = :rb, labelsize = 12)
fig